In [1]:
!pip install -q openai


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.0 MB/s eta 0:00:00


In [2]:
#import openai
import os
from openai import OpenAI
from google.colab import userdata


key = userdata.get('OLSAI_API')

client = OpenAI(
    # This is the default and can be omitted
    api_key = key
)


In [4]:
# Text completion
def my_ai_assistant(prompt):
    response = client.chat.completions.create(
       messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
    model="gpt-4-turbo",
)
    return response.choices[0].message

# Example usage
prompt = "Translate the following English text to French: 'Hello, how are you?'"
result = my_ai_assistant(prompt)
print("My AI's response:", result)

My AI's response: ChatCompletionMessage(content='Bonjour, comment vas-tu ?', role='assistant', function_call=None, tool_calls=None)


In [ ]:
# Function to upload a dataset
from pathlib import Path

file = client.files.create(
    file =  Path('sample_data/cacao.csv'),
    purpose='assistants'
)
file.id

'file-gYOAHeB6gq7d2wcHtsuGqzGW'

In [ ]:
# Create an assistant using the file ID
assistant = client.beta.assistants.create(
  instructions="OLSAI",
  model="gpt-4-turbo",
  tools=[{"type": "code_interpreter"}],
  tool_resources={
    "code_interpreter": {
      "file_ids": [file.id]
    }
  }
)

In [ ]:
# Create an empty thread
thread = client.beta.threads.create()
  #messages=[
   # {
    #  "role": "user",
     # "content": "List summary statistics",
      #"attachments": [
       # {
        #  "file_id": file.id,
         # "tools": [{"type": "code_interpreter"}]
        #}
      #]
    #}

  #]
#)


In [ ]:
thread_message = client.beta.threads.messages.create(
  thread.id,
  role="user",
  content="Make summary statistics",
  attachments=[
        {
          "file_id": file.id,
          "tools": [{"type": "code_interpreter"}]
        }]
)
print(thread_message)

Message(id='msg_U34DGRaXeqQsDwu0C6BIdv13', assistant_id=None, attachments=[Attachment(file_id='file-gYOAHeB6gq7d2wcHtsuGqzGW', tools=[CodeInterpreterTool(type='code_interpreter')])], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Make summary statistics'), type='text')], created_at=1714914334, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_ZIOQpBr02IOqTF86muKmMVOc')


In [ ]:
run = client.beta.threads.runs.create_and_poll(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions="Please address the user as Student."
)

In [ ]:
if run.status == 'completed':
  messages = client.beta.threads.messages.list(
    thread_id=thread.id
  )
  print(messages)
else:
  print(run.status)

SyncCursorPage[Message](data=[Message(id='msg_5s9FdJ8mk3WaNEdOG3gpeBwo', assistant_id='asst_oPHhEA2KvLU5gaKRkJGElU2e', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Here are the summary statistics for each variable in the dataset:\n\n- **ant_exclusion:** Binary (0 or 1). Mean indicates a balanced presence of both classes.\n- **stem_diameter:** Average diameter is approximately 27.1, with a standard deviation of 5.3, indicating moderate variability in stem diameter.\n- **height:** Plants have an average height of 293, with a standard deviation of 34.6, showing some variation in plant heights.\n- **canopy:** The mean canopy width is 0.33, with values ranging up to 0.71.\n- **dw_healthy:** Mean dry weight of healthy parts is 1054.78, but with high variability (std 750.21).\n- **dw_infect:** Mean dry weight of infected parts is 149.55, similar variability to healthy parts (std 146.95).\n- **dw_total:** Total dry weight has a mean of 1203.53, 

In [ ]:
messages.data[0].content

[TextContentBlock(text=Text(annotations=[], value='Here are the summary statistics for each variable in the dataset:\n\n- **ant_exclusion:** Binary (0 or 1). Mean indicates a balanced presence of both classes.\n- **stem_diameter:** Average diameter is approximately 27.1, with a standard deviation of 5.3, indicating moderate variability in stem diameter.\n- **height:** Plants have an average height of 293, with a standard deviation of 34.6, showing some variation in plant heights.\n- **canopy:** The mean canopy width is 0.33, with values ranging up to 0.71.\n- **dw_healthy:** Mean dry weight of healthy parts is 1054.78, but with high variability (std 750.21).\n- **dw_infect:** Mean dry weight of infected parts is 149.55, similar variability to healthy parts (std 146.95).\n- **dw_total:** Total dry weight has a mean of 1203.53, with a standard deviation of 824.98, indicating broad variation.\n- **fw_pulb:** Fresh weight of pulp averages 16239.71 with a significant standard deviation of 1

In [ ]:
run_steps = client.beta.threads.runs.steps.list(
    thread_id=thread.id,
    run_id=run.id
)

for step in run_steps.data:

  # If step involves code execution, print the code
  if step.type == 'code_interpreter':
      print(f"Python Code Executed: {step.step_details['code_interpreter']['input']}")
